## Chapter 7: Externalizing Configuration with Aspire

In a monolithic application, configuration often lives in files like `appsettings.json`. But as you move to distributed, cloud‑native architectures, this approach breaks down. You need to manage configuration across multiple services, support different environments (development, staging, production), and handle secrets securely. Hard‑coding connection strings or feature flags in source code is no longer acceptable.

.NET Aspire provides components that integrate with **Azure App Configuration** and **Azure Key Vault** to externalize configuration and secrets. These services offer:

- **Centralized configuration**: Manage settings for all services in one place.
- **Dynamic updates**: Change configuration without restarting services.
- **Secret management**: Store sensitive values securely, with access policies.
- **Feature flags**: Toggle features on/off without redeploying.

In this chapter, you’ll learn how to add Azure App Configuration and Key Vault to your Aspire application, how to use their respective components, and how to manage secrets locally using Aspire’s parameter resources. By the end, your e‑commerce application will be ready for production‑grade configuration management.

---

### 7.1 The Limits of appsettings.json

Consider our `MyAspireApp` solution. The API service’s `appsettings.json` might contain:

```json
{
  "Logging": { ... },
  "FeatureFlags": {
    "EnableNewCheckout": false
  },
  "ConnectionStrings": {
    "productsdb": "Host=localhost;Port=5432;Database=shop;Username=postgres;Password=..."
  }
}
```

Problems with this approach:

- The database password is in plain text (if checked into source control).
- The feature flag cannot be changed without restarting the service.
- Different services may need different settings, leading to duplication.
- In production, the connection string and other secrets must be injected via environment variables or other means, which is error‑prone.

Aspire’s configuration components solve these problems by integrating with cloud‑grade services.

---

### 7.2 .NET Aspire Configuration Components

Aspire provides two main components for external configuration:

- **`Aspire.Azure.Data.AppConfiguration`**: Connects to Azure App Configuration, providing access to key‑value pairs, feature flags, and configuration revision history.
- **`Aspire.Azure.KeyVault`**: Connects to Azure Key Vault, allowing you to retrieve secrets securely.

Both components follow the same pattern as other Aspire components: they read the connection string from configuration (injected by the AppHost), register the appropriate clients, and add health checks and telemetry.

---

### 7.3 Setting Up Azure App Configuration

#### 7.3.1 Adding App Configuration to the AppHost

First, add the hosting package to the AppHost project:

```bash
cd MyAspireApp.AppHost
dotnet add package Aspire.Hosting.Azure.AppConfiguration
```

Now you can add an App Configuration resource. For local development, you have two options:

- Use a connection string to a real Azure App Configuration instance.
- Use the **Azure App Configuration emulator** (if available). Currently, there is no official emulator, but you can use a connection string to a real instance even in development (or use a test instance). Alternatively, you can use the `RunAsEmulator` method if the emulator exists; we'll assume we use a real instance for simplicity.

```csharp
// In AppHost/Program.cs
var appConfig = builder.AddAzureAppConfiguration("appconfig")
    .WithEndpoint("https://myappconfig.azconfig.io")  // Optional: specify endpoint
    .WithAccessKey("credential");                      // Optional: if not using managed identity
```

For development, you might use a connection string that includes the endpoint and access key. Aspire’s hosting package can generate these for you if you use `AddConnectionString` or parameters. A simpler way is to use a parameter for the connection string.

We'll use a parameter to supply the connection string securely:

```csharp
var appConfigConnectionString = builder.AddParameter("appconfig-connectionstring", secret: true);
var appConfig = builder.AddAzureAppConfiguration("appconfig")
    .WithConnectionString(appConfigConnectionString);
```

Then, when you run the AppHost, you’ll be prompted for the connection string, or you can store it in user secrets.

Alternatively, you can use the `AddAzureAppConfiguration` overload that accepts a connection string directly from configuration:

```csharp
var appConfig = builder.AddAzureAppConfiguration("appconfig");
```

This expects the connection string to be in configuration under `ConnectionStrings:appconfig`. You can set that via user secrets or environment variables.

Now reference this resource from your services:

```csharp
var apiService = builder.AddProject<Projects.MyAspireApp_ApiService>("apiservice")
    .WithReference(appConfig)
    .WithReference(productsDb)
    .WithReference(messaging);
```

#### 7.3.2 Using the App Configuration Component in a Service

In the API service project, add the component package:

```bash
cd ../MyAspireApp.ApiService
dotnet add package Aspire.Azure.Data.AppConfiguration
```

In `Program.cs`, add the App Configuration client:

```csharp
builder.AddAzureAppConfigurationClient("appconfig");
```

This registers an `ConfigurationClient` from the Azure SDK. You can then use it to read configuration values, but more commonly, you’ll use the **ASP.NET Core configuration provider** integration. The component actually registers an `IConfigurationSource` that pulls from App Configuration and integrates with the .NET configuration system. However, the `AddAzureAppConfigurationClient` method only registers the client; for full integration, you need to call `AddAzureAppConfiguration` on the `ConfigurationBuilder`. Let’s do that properly.

The Aspire component documentation suggests using `builder.Configuration.AddAzureAppConfiguration` after building the builder, but because we’re using the generic host, we can do:

```csharp
using Azure.Identity;
using Azure.Data.AppConfiguration;

var builder = WebApplication.CreateBuilder(args);

// Add service defaults
builder.AddServiceDefaults();

// Add Azure App Configuration
builder.Configuration.AddAzureAppConfiguration(options =>
{
    var connectionString = builder.Configuration.GetConnectionString("appconfig");
    options.Connect(connectionString)
           .Select("*")  // load all keys
           .ConfigureRefresh(refresh =>
           {
               refresh.Register("Sentinel", refreshAll: true)
                      .SetCacheExpiration(TimeSpan.FromMinutes(5));
           });
});

// Add feature management if you use feature flags
builder.Services.AddFeatureManagement();
```

But wait—we also have the component’s client registration. Actually, the `AddAzureAppConfigurationClient` method registers the `ConfigurationClient` for direct use, but for configuration integration, we need the `Microsoft.Azure.AppConfiguration.AspNetCore` package. Aspire’s component is primarily for registering the client and adding health checks. For full configuration integration, you still need to call `AddAzureAppConfiguration` on the configuration builder. The component’s `AddAzureAppConfigurationClient` does not automatically add the configuration provider; it just registers the client for dependency injection.

Let's clarify: The `Aspire.Azure.Data.AppConfiguration` package provides an extension method `AddAzureAppConfigurationClient` that registers a `ConfigurationClient` in DI and adds health checks. It does **not** automatically add the configuration provider. To load configuration from App Configuration into the .NET configuration system, you need to use the `Microsoft.Azure.AppConfiguration.AspNetCore` package and call `builder.Configuration.AddAzureAppConfiguration`. The Aspire component is useful if you want to manually query App Configuration, but for most scenarios, you want the configuration provider.

We'll cover both: using the provider for seamless configuration, and using the client for advanced scenarios.

**Option 1: Using the configuration provider (recommended)**

Install the required package:

```bash
dotnet add package Microsoft.Azure.AppConfiguration.AspNetCore
```

Then modify `Program.cs`:

```csharp
using Azure.Identity;

var builder = WebApplication.CreateBuilder(args);

builder.AddServiceDefaults();

// Add Azure App Configuration to the configuration system
builder.Configuration.AddAzureAppConfiguration(options =>
{
    var connectionString = builder.Configuration.GetConnectionString("appconfig");
    options.Connect(connectionString)
           .Select("*")  // load all keys
           .ConfigureRefresh(refresh =>
           {
               // Use a sentinel key to trigger a full refresh
               options.Register("Sentinel", refreshAll: true)
                      .SetCacheExpiration(TimeSpan.FromMinutes(5));
           });
});

// Add feature management (if using feature flags)
builder.Services.AddFeatureManagement();

// ... rest of your services
```

Now any configuration key from App Configuration will be available via `IConfiguration`. You can even enable dynamic refresh.

**Option 2: Using the client directly**

If you need to query App Configuration programmatically (e.g., for administrative tools), you can inject `ConfigurationClient`. But for normal configuration, the provider is simpler.

#### 7.3.3 Dynamic Configuration Reload

One of the key benefits of App Configuration is the ability to change settings without restarting the application. To enable dynamic reload, you need to:

1. Configure refresh in the options (as shown above with `ConfigureRefresh`).
2. Use the `Microsoft.FeatureManagement` library to watch for changes.
3. Call `options.UseFeatureFlags()` if using feature flags.
4. In your code, inject `IConfigurationRefresher` or use the middleware.

First, add the feature management package:

```bash
dotnet add package Microsoft.FeatureManagement.AspNetCore
```

Then update the App Configuration setup:

```csharp
builder.Configuration.AddAzureAppConfiguration(options =>
{
    var connectionString = builder.Configuration.GetConnectionString("appconfig");
    options.Connect(connectionString)
           .Select("*")
           .ConfigureRefresh(refreshOptions =>
           {
               refreshOptions.Register("Sentinel", refreshAll: true)
                             .SetCacheExpiration(TimeSpan.FromMinutes(5));
           })
           .UseFeatureFlags();  // enables feature flags
});

builder.Services.AddFeatureManagement();
```

To trigger a refresh, you can inject `IConfigurationRefresher` and call `TryRefreshAsync` periodically, or use the middleware that automatically refreshes on requests. For simplicity, we'll add the middleware:

```csharp
app.UseAzureAppConfiguration();
```

This middleware will check for configuration changes on each request (within the cache expiration). Now if you change a value in App Configuration, the application will pick it up after the cache expires.

#### 7.3.4 Using Feature Flags

Feature flags are a powerful way to control feature availability. With App Configuration and the `Microsoft.FeatureManagement` library, you can define flags and toggle them without redeploying.

First, define a feature flag in App Configuration (e.g., "NewCheckout"). Then in your code:

```csharp
[ApiController]
[Route("api/checkout")]
public class CheckoutController : ControllerBase
{
    private readonly IFeatureManager _featureManager;

    public CheckoutController(IFeatureManager featureManager)
    {
        _featureManager = featureManager;
    }

    [HttpPost]
    public async Task<IActionResult> Checkout(OrderRequest request)
    {
        if (await _featureManager.IsEnabledAsync("NewCheckout"))
        {
            // Use new checkout logic
        }
        else
        {
            // Use old checkout logic
        }
        return Ok();
    }
}
```

The feature flag value is evaluated dynamically, and changes are reflected after the refresh interval.

---

### 7.4 Azure Key Vault for Secrets

Azure Key Vault provides secure storage for secrets like passwords, connection strings, and API keys. Aspire’s Key Vault component makes it easy to integrate.

#### 7.4.1 Adding Key Vault to the AppHost

Add the hosting package:

```bash
cd MyAspireApp.AppHost
dotnet add package Aspire.Hosting.Azure.KeyVault
```

Add a Key Vault resource:

```csharp
var keyVault = builder.AddAzureKeyVault("keyvault")
    .WithReference(appConfig);  // optional: Key Vault can be referenced by App Configuration
```

But Key Vault itself doesn’t run locally; you need a real Azure Key Vault or use the emulator? There is no emulator for Key Vault, so you'll need to use a real instance, even in development. You can create a free tier Key Vault in Azure and use it for development, or use a parameter for the URI and credentials.

Typically, you'd reference Key Vault from your services so they can retrieve secrets directly. However, a common pattern is to store secrets in Key Vault and have App Configuration reference them. This way, App Configuration acts as a unified configuration layer, and Key Vault holds the actual secret values. The App Configuration service can be granted access to Key Vault and automatically resolve secret references.

We'll set up Key Vault and then configure App Configuration to reference secrets.

In the AppHost, we can model both:

```csharp
var keyVault = builder.AddAzureKeyVault("keyvault")
    .WithUri("https://myvault.vault.azure.net/");

var appConfig = builder.AddAzureAppConfiguration("appconfig")
    .WithEndpoint("https://myappconfig.azconfig.io")
    .WithAccessKey("...")   // or use managed identity
    .WithKeyVaultReferences(keyVault);  // this tells App Configuration it can resolve secrets from this Key Vault
```

The `WithKeyVaultReferences` method is conceptual; the actual hosting package may not have it yet. Instead, you would configure this relationship in Azure itself. For local development, you might not need the Key Vault reference; you could inject secrets directly via environment variables or parameters.

#### 7.4.2 Using the Key Vault Component in a Service

Add the component package to the API service:

```bash
cd ../MyAspireApp.ApiService
dotnet add package Aspire.Azure.Security.KeyVault
```

In `Program.cs`:

```csharp
builder.AddAzureKeyVaultClient("keyvault");
```

This registers `SecretClient` and `KeyClient` in DI. You can then inject `SecretClient` to retrieve secrets manually. But again, you might prefer to integrate with .NET configuration using the Azure Key Vault configuration provider.

The `Microsoft.Extensions.Configuration.AzureKeyVault` package allows you to add Key Vault as a configuration source. Aspire’s component does not automatically do that; it only registers the client. So you need to install:

```bash
dotnet add package Microsoft.Extensions.Configuration.AzureKeyVault
```

Then in `Program.cs`, after building the builder but before building the app, you can add:

```csharp
using Azure.Identity;

builder.Configuration.AddAzureKeyVault(
    new Uri(builder.Configuration.GetConnectionString("keyvault")), 
    new DefaultAzureCredential());
```

This will load all secrets from Key Vault as configuration values (with the secret name as the key). However, in practice, you often reference secrets via App Configuration, so you don't need to add Key Vault directly to configuration; App Configuration will resolve them.

#### 7.4.3 Combining App Configuration and Key Vault

The recommended pattern is:

- Store non‑secret configuration in App Configuration.
- Store secrets in Key Vault.
- In App Configuration, create entries that reference Key Vault secrets (using the `{uri}` syntax). For example, a key `Database:Password` might have a value that is a reference to a Key Vault secret.
- The App Configuration provider automatically resolves these references, so your application sees the actual secret value.

To enable this, when you configure the App Configuration provider, you need to provide a `KeyVaultCredential` and `SecretClient` or configure it to use managed identity. With the Aspire component, you might still need to set up the credential.

In `Program.cs`, you can do:

```csharp
builder.Configuration.AddAzureAppConfiguration(options =>
{
    var connectionString = builder.Configuration.GetConnectionString("appconfig");
    options.Connect(connectionString)
           .Select("*")
           .ConfigureKeyVault(kv =>
           {
               kv.SetCredential(new DefaultAzureCredential());
           })
           .ConfigureRefresh(...)
           .UseFeatureFlags();
});
```

This tells the provider to resolve Key Vault references using `DefaultAzureCredential`.

Now, when your application reads `Database:Password`, it will get the actual secret from Key Vault.

---

### 7.5 Managing Secrets Locally with Parameters and User Secrets

Even with external configuration, you still need to provide connection strings for the external services themselves (e.g., the App Configuration connection string). These are secrets that should not be in source code. Aspire’s **parameter resources** are designed for this.

In the AppHost, you can define parameters:

```csharp
var appConfigConnectionString = builder.AddParameter("appconfig-connectionstring", secret: true);
var appConfig = builder.AddAzureAppConfiguration("appconfig")
    .WithConnectionString(appConfigConnectionString);
```

When you run the AppHost, if the parameter value is not found, you’ll be prompted to enter it. You can also store it in user secrets:

```bash
dotnet user-secrets init --project MyAspireApp.AppHost
dotnet user-secrets set "Parameters:appconfig-connectionstring" "<your-connection-string>"
```

This keeps secrets out of the code and out of configuration files that might be accidentally committed.

For the API service’s own secrets (like the database password), you might want to keep them in Key Vault, but during development you could also use user secrets. The API service has its own user secrets, but those are not managed by the AppHost. The AppHost only injects configuration via environment variables, so you can also set environment variables for the service.

A good practice: use App Configuration + Key Vault for all environment‑specific settings, and use parameters for the credentials needed to access those services. Then the AppHost injects the connection strings to App Configuration, and the service fetches everything else from there.

---

### 7.6 Hands‑on: Move Configuration and Secrets to Azure

Now let’s apply these concepts to our e‑commerce application. We’ll:

1. Create an Azure App Configuration instance (free tier) and an Azure Key Vault (free tier) in the Azure portal.
2. Store some configuration values and a feature flag.
3. Store the database password as a secret in Key Vault and reference it from App Configuration.
4. Update the API service to use external configuration.
5. Test dynamic reload.

#### Step 1: Create Azure Resources

- Create a Resource Group.
- Create an App Configuration (choose the free tier).
- Create a Key Vault (choose the standard tier, or use the free tier if available).
- In App Configuration, add a key `Sentinel` with a value like `1` (used for refresh).
- Add a feature flag named `NewCheckout` with initial state `false`.
- In Key Vault, add a secret named `productsdb-password` with the actual database password.
- In App Configuration, add a key `Database:Password` with value `{https://myvault.vault.azure.net/secrets/productsdb-password}` (the secret reference URI). Also add other configuration like `Database:Host`, `Database:Port` etc., but those are not secrets.
- Grant your user (or the application’s managed identity) access to Key Vault (Secret Get permission) and to App Configuration (Data Reader role). For local development, you’ll likely use your own credentials via `DefaultAzureCredential`.

#### Step 2: Update AppHost with Parameters and References

In AppHost `Program.cs`, add parameters for the App Configuration connection string and possibly Key Vault URI. For simplicity, we'll use connection string for App Configuration and assume the Key Vault URI is known.

```csharp
var appConfigConnectionString = builder.AddParameter("appconfig-connectionstring", secret: true);
var appConfig = builder.AddAzureAppConfiguration("appconfig")
    .WithConnectionString(appConfigConnectionString);

// Add Key Vault resource (for reference only, not directly used by services)
var keyVault = builder.AddAzureKeyVault("keyvault")
    .WithUri("https://myvault.vault.azure.net/");  // Hard-coded or parameter

apiService.WithReference(appConfig);
// No direct reference to Key Vault from API; API will get secrets via App Configuration
```

#### Step 3: Update API Service to Use App Configuration Provider

Install the necessary packages in the API project:

```bash
dotnet add package Microsoft.Azure.AppConfiguration.AspNetCore
dotnet add package Microsoft.FeatureManagement.AspNetCore
dotnet add package Azure.Identity
```

Modify `Program.cs`:

```csharp
using Azure.Identity;
using Microsoft.FeatureManagement;

var builder = WebApplication.CreateBuilder(args);

builder.AddServiceDefaults();

// Add Azure App Configuration
builder.Configuration.AddAzureAppConfiguration(options =>
{
    var connectionString = builder.Configuration.GetConnectionString("appconfig");
    options.Connect(connectionString)
           .Select("*")
           .ConfigureKeyVault(kv =>
           {
               kv.SetCredential(new DefaultAzureCredential());
           })
           .ConfigureRefresh(refresh =>
           {
               refresh.Register("Sentinel", refreshAll: true)
                      .SetCacheExpiration(TimeSpan.FromMinutes(1));  // short for demo
           })
           .UseFeatureFlags();
});

builder.Services.AddFeatureManagement();

// Now, the database connection string can be built from configuration
// The password will come from Key Vault via App Configuration
var dbHost = builder.Configuration["Database:Host"] ?? "localhost";
var dbPort = builder.Configuration["Database:Port"] ?? "5432";
var dbName = builder.Configuration["Database:Name"] ?? "shop";
var dbUser = builder.Configuration["Database:User"] ?? "postgres";
var dbPassword = builder.Configuration["Database:Password"]; // from Key Vault

var connectionString = $"Host={dbHost};Port={dbPort};Database={dbName};Username={dbUser};Password={dbPassword}";
builder.Services.AddNpgsqlDataSource(connectionString);

// ... rest of services (RabbitMQ, etc.)

var app = builder.Build();

app.UseAzureAppConfiguration();  // middleware for dynamic refresh

// ... rest of pipeline

app.Run();
```

Note: We no longer use `builder.AddNpgsqlDataSource("productsdb")` because we're building the connection string manually from configuration. But you could also register the data source with that connection string.

#### Step 4: Test Dynamic Configuration

Run the AppHost. The first time, you'll be prompted for the App Configuration connection string. Enter it (or set in user secrets). The API service should start successfully.

Now, go to the Azure portal and change the `NewCheckout` feature flag to `true`. Wait a minute (cache expiration) and then call an endpoint that uses the feature flag. You should see the new behavior without restarting.

Change the `Sentinel` value to trigger a refresh of all configuration. The database password could be rotated in Key Vault, and after refresh, the API would pick up the new password (though connections might need to be re-established).

#### Step 5: (Optional) Add Parameter for Key Vault URI

Instead of hard‑coding the Key Vault URI, you can parameterize it:

```csharp
var keyVaultUri = builder.AddParameter("keyvault-uri", secret: false);
var keyVault = builder.AddAzureKeyVault("keyvault")
    .WithUri(keyVaultUri.Resource);
```

Then set the URI via user secrets or environment.

---

### 7.7 Summary

In this chapter, you learned how to externalize configuration using Azure App Configuration and Azure Key Vault within an Aspire application. Key takeaways:

- **Azure App Configuration** provides a central place for configuration and feature flags, with dynamic refresh capabilities.
- **Azure Key Vault** securely stores secrets, and can be integrated with App Configuration to resolve secret references.
- **Aspire components** simplify connecting to these services, but for full configuration integration, you may need additional packages like `Microsoft.Azure.AppConfiguration.AspNetCore`.
- **Parameter resources** in the AppHost allow you to securely provide connection strings for these external services, using user secrets for local development.
- Dynamic configuration and feature flags enable you to change application behavior without redeploying.

By adopting external configuration, your application becomes more flexible, secure, and ready for production. In the next chapter, we’ll explore **Storage and Blobs** with Aspire. You’ll learn how to integrate Azure Blob Storage for file uploads, and how to use the emulator for local development. We’ll also cover how to manage large files and integrate with the rest of your e‑commerce application.

---

**Exercises**

1. Set up Azure App Configuration and Key Vault for your own Azure subscription. Move at least three configuration settings and one secret to these services.
2. Implement a feature flag in the web frontend (Blazor) that shows or hides a new UI element. Use the `Microsoft.FeatureManagement` library in Blazor.
3. Experiment with dynamic refresh: change a configuration value and observe the cache expiration behavior.
4. Instead of building the database connection string manually, create a custom configuration provider that constructs it from parts and registers `NpgsqlDataSource`. Compare the complexity.

In Chapter 8, we’ll dive into **Storage and Blobs**, adding image upload functionality to our e‑commerce site.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='6. messaging_and_event_driven_architecture.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='8. storage_and_blobs.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
